##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 通过 Keras 模型创建 Estimator

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://tensorflow.google.cn/tutorials/estimator/keras_model_to_estimator" class=""><img src="https://tensorflow.google.cn/images/tf_logo_32px.png" class="">在 TensorFlow.org 上查看</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/estimator/keras_model_to_estimator.ipynb" class=""><img src="https://tensorflow.google.cn/images/colab_logo_32px.png" class="">在 Google Colab 中运行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/estimator/keras_model_to_estimator.ipynb" class=""><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png" class="">在 GitHub 上查看源代码</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/estimator/keras_model_to_estimator.ipynb" class=""><img src="https://tensorflow.google.cn/images/download_logo_32px.png" class="">下载笔记本</a></td>
</table>

## 概述

TensorFlow 完全支持 TensorFlow Estimator，可以从新的和现有的 `tf.keras` 模型创建 Estimator。本教程包含了该过程完整且最为简短的示例。

## 设置

In [ ]:
import tensorflow as tf

import numpy as np
import tensorflow_datasets as tfds

### 创建一个简单的 Keras 模型。

在 Keras 中，需要通过组装*层*来构建*模型*。模型（通常）是由层构成的计算图。最常见的模型类型是一种叠加层：`tf.keras.Sequential` 模型。

构建一个简单的全连接网络（即多层感知器）：

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3)
])

编译模型并获取摘要。

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam')
model.summary()

### 创建输入函数

使用 [Datasets API](../../guide/data.md) 可以扩展到大型数据集或多设备训练。

Estimator 需要控制构建输入流水线的时间和方式。为此，它们需要一个“输入函数”或 `input_fn`。`Estimator` 将不使用任何参数调用此函数。`input_fn` 必须返回 `tf.data.Dataset`。

In [ ]:
def input_fn():
  split = tfds.Split.TRAIN
  dataset = tfds.load('iris', split=split, as_supervised=True)
  dataset = dataset.map(lambda features, labels: ({'dense_input':features}, labels))
  dataset = dataset.batch(32).repeat()
  return dataset

测试您的 `input_fn`

In [ ]:
for features_batch, labels_batch in input_fn().take(1):
  print(features_batch)
  print(labels_batch)

### 通过 tf.keras 模型创建 Estimator。

可以使用 `tf.estimator` API 来训练 `tf.keras.Model`，方法是使用 `tf.keras.estimator.model_to_estimator` 将模型转换为 `tf.estimator.Estimator` 对象。

In [ ]:
import tempfile
model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model, model_dir=model_dir)

训练和评估 Estimator。

In [ ]:
keras_estimator.train(input_fn=input_fn, steps=500)
eval_result = keras_estimator.evaluate(input_fn=input_fn, steps=10)
print('Eval result: {}'.format(eval_result))